In [16]:
import pandas as pd

# Analisis de trayectorio

Dataset de la informacion de paraderos (busstop.csv):
- Muestra informacion de los paraderos que se utilizan durante el trayectorio de los autobuses, tanto como la latitud y longitud.
- Se consideran puntos gps en formato de latitud y longitud


Se procesa la nueva trayectoria de paraderos y analizar los radios que se les asignaran para el geofencing, creando un nuevo archivo de la trayectoria de paraderos.

In [17]:
ida = pd.read_csv('../data/busstops_ida.csv', sep=',')
ida

,number,id,name,navigation,latitude,longitude,radio
0,0,0,inicio,159,-13.554978,-71.856025,30
1,1,36,primer stop,304,-13.554812,-71.857300,18
2,2,37,segundo stop,281,-13.553294,-71.861944,13
3,3,38,Ccollana,283,-13.552296,-71.867299,9
4,4,39,Puente Huaccoto,283,-13.551592,-71.870211,10
...,...,...,...,...,...,...,...
79,79,114,Segundo,221,-13.547528,-71.985248,10
80,80,115,Reservorio,225,-13.549660,-71.988003,10
81,81,116,Cuarto,230,-13.550408,-71.988880,10
82,82,117,Quinto,203,-13.550841,-71.989448,10


In [18]:
vuelta =  pd.read_csv('../data/busstop.csv', sep=',')


new_col = [0 for _ in range(vuelta.shape[0])]
vuelta['radio'] = new_col
vuelta = vuelta.reindex(columns=ida.columns)
vuelta = vuelta.tail(152-81)

In [19]:
df_final = pd.concat([ida, vuelta], ignore_index=True)
#df_final.to_csv('busstop_radio0.csv', sep=',')
df_final.drop(columns=['radio'], inplace=True)
df_final

,number,id,name,navigation,latitude,longitude
0,0,0,inicio,159,-13.554978,-71.856025
1,1,36,primer stop,304,-13.554812,-71.857300
2,2,37,segundo stop,281,-13.553294,-71.861944
3,3,38,Ccollana,283,-13.552296,-71.867299
4,4,39,Puente Huaccoto,283,-13.551592,-71.870211
...,...,...,...,...,...,...
150,146,183,Tapia,107,-13.549241,-71.878975
151,147,184,Forestal,107,-13.549985,-71.876345
152,148,185,Kayra,107,-13.550608,-71.874402
153,149,186,Puente,105,-13.551644,-71.870166


In [15]:
# Guarda el DataFrame modificado en un nuevo archivo CSV
df_final.to_csv('../data/busstop_radio0.csv', index=False)

In [5]:
# cargar datos en bruto
busstop_df = pd.read_csv('../data/busstop.csv', sep=',')
busstop_df


,number,id,name,latitude,longitude,navigation
0,0,38,Ccollana,-13.552296,-71.867299,283
1,1,39,Puente Huaccoto,-13.551592,-71.870211,283
2,2,40,Kayra,-13.550640,-71.873952,289
3,3,41,Forestal,-13.550020,-71.876104,286
4,4,42,Grifo Tapia,-13.549282,-71.878649,284
...,...,...,...,...,...,...
146,146,183,Tapia,-13.549241,-71.878975,107
147,147,184,Forestal,-13.549985,-71.876345,107
148,148,185,Kayra,-13.550608,-71.874402,107
149,149,186,Puente,-13.551644,-71.870166,105


Se crea un nuevo formato para el manejo de paraderos de la ruta, uniendo las columnas de longitud y latitud

In [24]:
# leer dataset busstop
busstopdf = pd.read_csv('../data/busstop_radio0.csv')

# diseño del df: creacion de latitude_longitude
busstopdf['latitude_longitude'] = busstopdf.apply(lambda row: (row['latitude'], row['longitude']), axis=1)
busstopdf.drop(columns=['number', 'latitude', 'longitude'], inplace=True)

# para fines practicos modificamos el id = index para mejor lectura
busstopdf['id'] = [f'{i+100}' for i in busstopdf.index]

# mostrar info y elementos
print(busstopdf.info())
busstopdf

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155 entries, 0 to 154
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  155 non-null    object
 1   name                155 non-null    object
 2   navigation          155 non-null    int64 
 3   latitude_longitude  155 non-null    object
dtypes: int64(1), object(3)
memory usage: 5.0+ KB
None


,id,name,navigation,latitude_longitude
0,100,inicio,159,"(-13.5549784, -71.856025)"
1,101,primer stop,304,"(-13.5548123, -71.8573001)"
2,102,segundo stop,281,"(-13.5532944, -71.8619442)"
3,103,Ccollana,283,"(-13.552296, -71.867299)"
4,104,Puente Huaccoto,283,"(-13.551592, -71.870211)"
...,...,...,...,...
150,250,Tapia,107,"(-13.549241, -71.878975)"
151,251,Forestal,107,"(-13.549985, -71.876345)"
152,252,Kayra,107,"(-13.550608, -71.874402)"
153,253,Puente,105,"(-13.551644, -71.870166)"


Generar los links de ida y vuelta

In [26]:
# creamos dataframe links y renombramos columnas
tmp1 = busstopdf[['id','name']].head(busstopdf.shape[0]-1).reset_index(drop=True)
# tmp1.rename(columns={'id':'ini_id', 'name':'ini_name'}, inplace=True)
tmp1.rename(columns={'id':'init_stop', 'name':'init_name'}, inplace=True)

tmp2 = busstopdf[['id','name']].tail(busstopdf.shape[0]-1).reset_index(drop=True)
# tmp2.rename(columns={'id':'end_id', 'name':'end_name'}, inplace=True)
tmp2.rename(columns={'id':'end_stop', 'name':'end_name'}, inplace=True)
linksdf = pd.concat([tmp1, tmp2], axis=1)

# creamos la columnas link
# linksdf['link'] = linksdf.apply(lambda row: f"{row['ini_id']}:{row['end_id']}", axis=1)
linksdf['link'] = linksdf.apply(lambda row: f"{row['init_stop']}:{row['end_stop']}", axis=1)
linksdf['id_link'] = linksdf.index
linksdf.insert(0, 'id_link', linksdf.pop('id_link'))
linksdf

,id_link,init_stop,init_name,end_stop,end_name,link
0,0,100,inicio,101,primer stop,100:101
1,1,101,primer stop,102,segundo stop,101:102
2,2,102,segundo stop,103,Ccollana,102:103
3,3,103,Ccollana,104,Puente Huaccoto,103:104
4,4,104,Puente Huaccoto,105,Kayra,104:105
...,...,...,...,...,...,...
149,149,249,Posta,250,Tapia,249:250
150,150,250,Tapia,251,Forestal,250:251
151,151,251,Forestal,252,Kayra,251:252
152,152,252,Kayra,253,Puente,252:253


a
<img src='graficos/trayectoria_bus_stops.png' width='600' height='300'>

# 2. Analisis de los registros de monitorizacion

Tareas a realizar:
- Filtrar los paraderos de vuelta (los que se usa y eliminar los que no se usan y que conincidan con el de ida)

Se da nuevo formato a la tabla, creando nuevas columnas y eliminando otras.

In [20]:
# cargar datos en bruto
monitoringdf = pd.read_csv('../data/_Process_monitoring__202303251137.csv', sep=',')

# Creacion del las columnas datetime y latitude_longitude
monitoringdf['datetime'] = monitoringdf['date'] + ' ' + monitoringdf['time']
monitoringdf['latitude_longitude'] = monitoringdf.apply(lambda row: (row['latitude'], row['longitude']), axis=1)

# eliminar las columnas latitude, longitude, time, velocity. Luego reordenar columnas
monitoringdf.drop(columns=['latitude', 'longitude','time'], inplace=True)
monitoringdf = monitoringdf[['id', 'date','datetime', 'latitude_longitude', 'lap', 'navigation', 'vehicle_id']]
monitoringdf

,id,date,datetime,latitude_longitude,lap,navigation,vehicle_id
0,13173398,2023-03-01,2023-03-01 05:10:11,"(-13.5495973, -71.8775099)",1,285,SJ39
1,13177467,2023-03-01,2023-03-01 06:04:51,"(-13.5245292, -71.9775272)",1,298,SJ34
2,13189674,2023-03-01,2023-03-01 07:22:21,"(-13.5341828, -71.9807088)",1,7,SJ29
3,13192729,2023-03-01,2023-03-01 07:39:04,"(-13.5511464, -71.9869284)",1,202,SJ21
4,13193272,2023-03-01,2023-03-01 07:41:52,"(-13.548642, -71.88101)",1,107,SJ34
...,...,...,...,...,...,...,...
1685065,17241845,2023-03-25,2023-03-25 08:50:24,"(-13.5254398, -71.9750786)",2,288,SJ24
1685066,17253753,2023-03-25,2023-03-25 10:01:56,"(-13.5511209, -71.9896213)",2,253,SJ16
1685067,17257644,2023-03-25,2023-03-25 10:26:10,"(-13.5432069, -71.8977847)",2,126,SJ24
1685068,17266468,2023-03-25,2023-03-25 11:14:26,"(-13.5327737, -71.9218652)",1,258,SJ43


# Cleaning

Mostramos la cantidad de datos, existen puntos que por error de lectura se leen dos veces, estos puntos se detectan si en las columnas datetime y latitud-longitud existen datos iguales, esto no debe suceder ya que el bus esta en constante movimiento como el tiempo. Como consecuencia se decide eliminar estos duplicados.

In [21]:
# DUPLICADOS
print(f'Shape dataset: {monitoringdf.shape}')

# Verificar y contar si hay puntos duplicados segun datetime y latitude_longitude
base = ['datetime', 'latitude_longitude']
twocols_dupli = monitoringdf.duplicated(subset=base)
print(f"Nro de duplicados en {base}: {twocols_dupli.sum()}")

# ============================ Solo para comprobacion ============================
# Verificar y contar si hay puntos duplicados segun latitude_longitude
# onecol_dupli =  monitoringdf['latitude_longitude'].duplicated()
# print(len(twocols_dupli), len(onecol_dupli))
# print(twocols_dupli.sum(), onecol_dupli.sum())
# print(abs(twocols_dupli.sum() - onecol_dupli.sum()))
# tmp = [ not x==y for x,y in zip(twocols_dupli, onecol_dupli)]
# data_tmp = monitoringdf.copy()
# data_tmp['dupli'] = tmp
# data_tmp[data_tmp['dupli'] == True]
# ================================================================================

# Eliminar columnas
monitoringdf.drop_duplicates(subset=base, inplace=True)
print(f"Duplicados eliminados => shape dataset: {monitoringdf.shape}")

Shape dataset: (1685070, 7)
Nro de duplicados en ['datetime', 'latitude_longitude']: 13116
Duplicados eliminados => shape dataset: (1671954, 7)


Comprobamos si existen casillas vacias (Nans) en todo el dataframe.

In [22]:
# NANs
# verificar si hay datos faltantes
print("Cantidad de NaNs:")
for col in monitoringdf:
    nans = monitoringdf[col].isna().sum()
    print(f"\tColumna {col}: {nans}")

Cantidad de NaNs:
	Columna id: 0
	Columna date: 0
	Columna datetime: 0
	Columna latitude_longitude: 0
	Columna lap: 0
	Columna navigation: 0
	Columna vehicle_id: 0


In [ ]:
from haversine import haversine as hvs, Unit
import numpy as np

# FUNCION: filtrar los puntos de ida
def idaPoints(df):
    end1_stop = {'latitude_longitude':(-13.5513636,-71.988304), 'navigation':75, 'radio':50}
    end2_stop = {'latitude_longitude':(-13.5511395,-71.986925), 'navigation':206, 'radio':50}
    for i, row in df.iterrows():
        if (itPassed(row, end1_stop) or itPassed(row, end2_stop)):
            break
    return df.head(i+1)

# FUNCION: ver si un punto paso por un stop
def itPassed(point, stop):
    nav = abs(point['navigation'] - stop['navigation'])
    dis = round(hvs(point['latitude_longitude'], stop['latitude_longitude'], unit=Unit.METERS), 2)
    return (nav<=45 and dis<=stop['radio'])

# FUNCION: 
def searchStopsPoints(row, stops, r_nav=45, r_dtec=60):
    # verificar la navegacion y el radio de distancia
    stops = stops[abs(stops['navigation'] - row['navigation']) <= r_nav]
    stops['dis'] = stops['latitude_longitude'].apply(lambda stop: round(hvs(row['latitude_longitude'], stop, unit=Unit.METERS), 2))
    stops = stops[stops['dis'] <= r_dtec]
    
    # preparar el output
    row['stop'] = str(stops.loc[stops['dis'].idxmin(), 'id']) if len(stops) != 0 else np.nan
    row['dis'] = float(stops.loc[stops['dis'].idxmin(), 'dis']) if len(stops) != 0 else 0    
    return row

# FUNCION: CALCULA EL TIEMPO EN SEG DE UN PUNTO CON SU ANTERIOR
def time_travel(row, df):
    if {row['init_stop'], row['end_stop']}.issubset(set(df.index)):
        # print('if')
        ti, te =  df.loc[row['init_stop'], 'datetime'], df.loc[row['end_stop'], 'datetime']
        row['date_time_init'] = ti
        row['date_time_end'] = te
        time = abs((ti - te).total_seconds())
        
        #row['time_travel'] =  time if time < 300 else np.nan
        row['time_travel'] =  time
        row['init_idp'], row['end_idp'] = df.loc[row['init_stop'], 'id'], df.loc[row['end_stop'], 'id']
        row['veh'] = df.loc[row['init_stop'], 'vehicle_id']
        row['lap'] = df.loc[row['init_stop'], 'lap']
        row['date'] = df.loc[row['init_stop'], 'date']

    else:
        # print('else')
        row['date_time_init'] = np.nan
        row['date_time_end'] = np.nan
        row['time_travel'] = np.nan
        row['init_idp'] = np.nan
        row['end_idp'] = np.nan
        row['veh'] = np.nan
        row['lap'] = np.nan
        row['date'] = np.nan
        
    return row